In [2]:
import numpy as np
import pytsmod as tsm
import soundfile as sf
import threading
from scipy.interpolate import interp1d

In [3]:
def split_file(x,sr,time,tempo):
    '''
    input:
    -x, sf data
    -sr, sample rate
    -time(seconds)
    -tempo(bpm)
    '''
    x_split = []
    scale = []
    temp = []
    time.append((x.shape[-1])/sr)
    for i in range(len(time)-1):
        i1 = int(time[i]*sr)
        i2 = int((time[i+1]*sr)+1)
        print(i1,i2)
        temp = x[:,i1:i2].copy()
        print(temp)
        x_split.append(temp)
        scale.append(120/tempo[i])
    return(x_split, scale)

def time_stretch(input_file, pairs):
    '''
    input:
    - input_file (wav file)
    - pairs (t, bpm)
    output:
    - x (soundfile data)
    - sr (sample rate)
    '''
    x,sr = sf.read(input_file) #gets data and sample rate
    x = x.T
    print("length:",len(x[0]),"sr:",sr,"time:",(len(x[0])/sr))
    print(x)
    #gets data from pair
    time = pairs[0]
    bpm = pairs[1]
    #insert starting data
    time.insert(0,0) 
    bpm.insert(0,120)

    x_split,scale = split_file(x,sr,time,bpm) #splits data into multiple chunks
    x_output = np.array([[],[]])
    for i in range(len(x_split)):
        print("period:",i)
        print(x_split[i],scale[i])
        x_output = np.concatenate((x_output,(tsm.hptsm(x_split[i],scale[i]))),axis = 1) #added each chunks scaled
        print("current length: ",(len(x_output[0])/sr))
    return (x_output,sr)

In [4]:
# filepath = 'C:\\Users\\Tima\\Desktop\\Companion-code\\beat_tempo_detection\\songs\\around_the_output.wav'
filepath = 'C:\\Users\\Tima\\Desktop\\Companion-code\\beat_tempo_detection\\songs\\trumpet.mp3'
output_file = 'C:\\Users\\Tima\\Desktop\\Companion-code\\tsm.wav'
pairs = [[2],[240]] #at 2 seconds change bpm to 60 then 5 seconds change to 240
x,sr = time_stretch(filepath,pairs) #perform time stretch using pair
sf.write(output_file,np.ravel(x),sr) #save file

length: 187776 sr: 48000 time: 3.912
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
0 96001
[[0.         0.         0.         ... 0.05275001 0.09318307 0.110966  ]
 [0.         0.         0.         ... 0.01341669 0.06233741 0.09311306]]
96000 187777
[[0.110966   0.10279018 0.08289626 ... 0.         0.         0.        ]
 [0.09311306 0.09734363 0.0820768  ... 0.         0.         0.        ]]
period: 0
[[0.         0.         0.         ... 0.05275001 0.09318307 0.110966  ]
 [0.         0.         0.         ... 0.01341669 0.06233741 0.09311306]] 1.0
current length:  2.000020833333333
period: 1
[[0.110966   0.10279018 0.08289626 ... 0.         0.         0.        ]
 [0.09311306 0.09734363 0.0820768  ... 0.         0.         0.        ]] 0.5
current length:  2.9560208333333335


The code below that is based on the TimeStretchThread() is a draft. The actual TimeStretchThread() that is needed to be used is below.

In [7]:
# class TimeStretchThread(threading.Thread):
#     def __init__(self, AThread, input_file, pairs, output_filepath):
#         threading.Thread.__init__(self)
#         self.input_file = input_file
#         self.pairs = pairs
#         self.output_filepath = output_filepath
#         self.stop_request = False

#     # def run(self):
#     #     while not self.stop_request:
#     #         x, sr = time_stretch(self.input_file, self.pairs)
#     #         sf.write(output_file,np.ravel(x),sr) 
#     def run(self):
#         while not self.stop_request:
#             data = self.AThread.get_n_samples(10000)
#             if data is not None and len(data) > 5000:
#                 # do timestretching using data
#                 x, sr = time_stretch(data, self.BeatThread.output)
#                 self.output = x
#             self.stop_request = True

In [15]:
class TimeStretchThread(threading.Thread):
    def __init__(self, AudioThread, BeatThread, input_file, pairs, output_filepath):
        threading.Thread.__init__(self)
        self.input_file = input_file
        self.pairs = pairs
        self.output_filepath = output_filepath
        self.stop_request = False
        self.AThread = AudioThread
        self.BeatThread = BeatThread
        self.output = None

    def run(self):
        while not self.stop_request:
            data = self.AThread.get_n_samples(10000)
            if data is not None and len(data) > 5000:
                # do timestretching using data
                x, sr = time_stretch(data, self.BeatThread.output)
                self.output = x
            self.stop_request = True

        sf.write(self.output_filepath, np.ravel(self.output), sr)

In [16]:
filepath = 'C:\\Users\\Tima\\Desktop\\Companion-code\\beat_tempo_detection\\songs\\trumpet.mp3'
output_file = 'C:\\Users\\Tima\\Desktop\\Companion-code\\tsm.wav'
# pairs = [[2,240]]  # Modify pairs to match the length of time and bpm
pairs = [[2, 3], [60, 120]] # times and bpms for 2 periods

# t = TimeStretchThread(filepath, pairs, output_file)
t = TimeStretchThread(audio_thread, beat_thread, filepath, pairs, output_file)
t.start()

NameError: name 'audio_thread' is not defined